In [1]:
import sys
sys.path.append("..")

## Load Dataset

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("jtz18/skin-lesion")

c:\Users\jonta\code\school\t8\DeepLearning\50.039-DL\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jonta\code\school\t8\DeepLearning\50.039-DL\venv\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for jtz18/skin-lesion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/jtz18/skin-lesion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Seeding

In [3]:
import torch
import numpy as np
import random
import os

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

set_seed(42)

## Init Wandb

In [4]:
import wandb

# Define sweep config
sweep_configuration = {
    "name": "segFormer-sweep",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "Test Dice Score"},
    "parameters": {
        "learning_rate": {"values": [0.0001, 0.001, 0.01, 0.1]},
        "batch_size": {"values": [1, 2, 4, 8, 16, 32, 64]},
        "epochs": {"values": [10]},
        "dropout_rate": {"values": [0, 0.1, 0.2, 0.3, 0.4, 0.5]},
        # "kernel_size": {"values": [3, 5, 7]},
    },
}

# Initialize sweep by passing in config.
sweep_id = wandb.sweep(sweep=sweep_configuration, project="unet-skin-lesion")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: l9fcs25g
Sweep URL: https://wandb.ai/jooz-cave/unet-skin-lesion/sweeps/l9fcs25g


In [5]:
# Hyperparameters etc.
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

DEVICE = device

NUM_WORKERS = 0

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256

PIN_MEMORY = True
LOAD_MODEL = False
CHECKPOINT_FILENAME = ""   # Model checkpoint filename if LOAD is True eg: checkpoints/checkpoint_9.pth.tar or None
CLASS = "task1"

# Define Training Function

In [6]:
from tqdm import tqdm
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
            wandb.log({"Training Loss": loss})

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

## Augmentations

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn
from src.utils import *


train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

## Training Loop

In [8]:
from src.segFormer import Segformer
from src.maluNet import MALUNet
from src.unet import UNET

def main():
    run = wandb.init()

    # note that we define values from `wandb.config`
    # instead of defining hard values
    LEARNING_RATE = wandb.config.learning_rate
    BATCH_SIZE = wandb.config.batch_size
    NUM_EPOCHS = wandb.config.epochs
    DROPOUT_RATE = wandb.config.dropout_rate

    model = Segformer(dropout=DROPOUT_RATE).to(device)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    wandb.log({"MODEL": model.__class__.__name__})


    train_loader, val_loader, test_loader = get_loaders(
        dataset,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    start_epoch = 0
    if LOAD_MODEL:
        checkpoint = torch.load(CHECKPOINT_FILENAME)
        model, optimizer, start_epoch = load_checkpoint(checkpoint, model, optimizer)



    check_accuracy(val_loader, model, device=DEVICE, loss_fn=loss_fn)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(start_epoch, NUM_EPOCHS):
        print(f"epoch: {epoch}")
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
            "epoch": epoch,
        }
        save_checkpoint(checkpoint, filename=f"checkpoint_{epoch}.pth.tar")

        # check accuracy
        check_accuracy(val_loader, model, device=DEVICE, loss_fn=loss_fn)

        # print some examples to a folder
        save_predictions_as_imgs(
            val_loader, model, folder=f"saved_images/{epoch}", device=DEVICE
        )

    # Final Evaluation Test
    check_accuracy(test_loader, model, device=DEVICE, loss_fn=loss_fn, mode="test")
    run.finish()

In [9]:
wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: ftmchblt with config:
wandb: 	batch_size: 1
wandb: 	dropout_rate: 0.3
wandb: 	epochs: 10
wandb: 	learning_rate: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtz18 (jooz-cave). Use `wandb login --relogin` to force relogin


Validation Loss: 0.8194209432601929
Got 4795034/6553600 with acc 73.17
Dice score: 0.0
epoch: 0


100%|██████████| 2594/2594 [08:56<00:00,  4.83it/s, loss=nan]


=> Saving checkpoint
Validation Loss: 1.0449258840084077
Got 1758566/6553600 with acc 26.83
Dice score: 0.38626909255981445
epoch: 1


100%|██████████| 2594/2594 [08:45<00:00,  4.94it/s, loss=nan]


...

100%|██████████| 41/41 [06:16<00:00,  9.18s/it, loss=0.168]


=> Saving checkpoint
Validation Loss: 0.6714895367622375
Got 5848997/6553600 with acc 89.25
Dice score: 0.7681164741516113
epoch: 3


100%|██████████| 41/41 [06:18<00:00,  9.24s/it, loss=0.306]


=> Saving checkpoint
Validation Loss: 0.6581641435623169
Got 5904906/6553600 with acc 90.10
Dice score: 0.7601460218429565
epoch: 4


 27%|██▋       | 11/41 [01:46<05:17, 10.58s/it, loss=0.244]